# Data Pre-processing 

#### Team members:  Yaxiong Luo, Yueran Liang, Shengzhao Xia
#### Data-set: [Amazon meta data and review data](http://jmcauley.ucsd.edu/data/amazon/)

In [2]:
%matplotlib inline
import gzip
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
DATA_FOLDER = "./data/"
META_HEALTH_CARE = "meta_Health_and_Personal_Care.json.gz"
REVIEWS_HEALTH_CARE = "reviews_Health_and_Personal_Care.json.gz"

### 1. Functions 
 
1.1 Functions for loading data

Read the data into a **pandas dataframe** by using the two functions below. These two functions are provided by the website of our Amazon dataset.

In [6]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

In [7]:
def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

1.2 Functions for checking Nan value

We define the two functions below to verify if there exists nan value in the chosen column.

In [3]:
def checkNanValue(dataframe, column):
    print("Check if column {} exists Nan value: {}".format(column,dataframe[column].isnull().values.any()))
    return dataframe[column].isnull().values.any()
    
def checkDataframeNanValue(dataframe):
    list_Nancolumns = []
    list_columns = list(dataframe.columns)
    for column in list_columns:
        if checkNanValue(dataframe, column):
             list_Nancolumns.append(column)
    return list_Nancolumns

### 2. Health & Care metadata

After observing the Health & Care metadata, we discover that there are lots of null values and some columns we are not interested. We decide to do some data cleaning work 

2.1 Import Health & Care metadata

In [12]:
df_health_meta = getDF(DATA_FOLDER + META_HEALTH_CARE)
org_health_meta_df = df_health_meta.copy()
df_health_meta.head()

,asin,description,title,imUrl,related,salesRank,categories,price,brand
0,0077614992,This is an example product description.,Principles of Mgmt + Oper-CSUF Custom C,http://ecx.images-amazon.com/images/I/51G%2BRq...,"{'also_bought': ['0471730726', '0132834871', '...",{'Health & Personal Care': 168429},[[Health & Personal Care]],NaN,NaN
1,0615208479,By now we all know the benefits of exercise fo...,Brain Fitness Exercises Software,http://ecx.images-amazon.com/images/I/41kbZB04...,NaN,{'Health & Personal Care': 1346973},"[[Health & Personal Care, Personal Care]]",NaN,NaN
2,0615269990,What's wrong with your patient?Do all the symp...,Occam's Razor,http://ecx.images-amazon.com/images/I/51fH-ABe...,"{'also_bought': ['1935660152', '0071743979', '...",{'Toys & Games': 110575},"[[Health & Personal Care, Personal Care, Shavi...",34.99,NaN
3,0615315860,NaN,101 BlenderBottle Recipes Quick and Easy,http://ecx.images-amazon.com/images/I/21zOQu2Q...,"{'also_bought': ['B006VT9RBM', 'B0010JLMO8', '...",{'Health & Personal Care': 254068},[[Health & Personal Care]],NaN,NaN
4,0615406394,This is an example product description.,"Aphrodite Reborn - Women's Stories of Hope, Co...",http://ecx.images-amazon.com/images/I/51rJLgsi...,"{'also_bought': ['0966035232', '1421407205']}",{'Health & Personal Care': 377936},[[Health & Personal Care]],NaN,NaN


2.2  Discard several uninterested columns

We discard **description & imUrl** 

In [10]:
df_health_meta = df_health_meta.drop(['description','imUrl'],axis=1)

2.3  Check if there exits NaN Value in the DataFrame

In [11]:
list_Nancolumns = checkDataframeNanValue(df_health_meta)

Check if column asin exists Nan value: False
Check if column title exists Nan value: True
Check if column related exists Nan value: True
Check if column salesRank exists Nan value: True
Check if column categories exists Nan value: False
Check if column price exists Nan value: True
Check if column brand exists Nan value: True


2.4  Replace Nan value by 0

According to the result above, we know that some columns exists Nan value. Thus, we decide to only replace column 'price '**Nan** values as **0**. As for NAN value in other columns ( 'brand','salesRank' and 'related' ), we will process them later according to the data analysis.

Before we replace 'price' value, we check if it contains 0 at first.

In [10]:
df_health_meta[df_health_meta['price']==0]

,asin,description,title,imUrl,related,salesRank,categories,price,brand


It seems that we can replace with 0 without any ambiguity.

In [13]:
df_health_meta['price'] = df_health_meta['price'].fillna(0)

2.5 Only keep relevant products and discard all the products of other categories

We put emphasis on Health & Personal Care products.
Therefore, we **drop all columns that is not relevant with Health & Personal Care** from the dataset with the help of the attribute **category**.

In [14]:
# View the categories in Health & Care meta data
pd.set_option('max_colwidth',100)
df_health_meta['categories'][:10]

0                                                                             [[Health & Personal Care]]
1                                                              [[Health & Personal Care, Personal Care]]
2                      [[Health & Personal Care, Personal Care, Shaving & Hair Removal, Manual Shaving]]
3                                                                             [[Health & Personal Care]]
4                                                                             [[Health & Personal Care]]
5                                                                             [[Health & Personal Care]]
6                                   [[Health & Personal Care, Personal Care, Eye Care, Reading Glasses]]
7    [[Health & Personal Care, Medical Supplies & Equipment, Daily Living Aids, Visual Impairment Aid...
8    [[Health & Personal Care, Medical Supplies & Equipment, Daily Living Aids, Low Strength Aids, Gr...
9         [[Health & Personal Care, Stationery & Party 

In [15]:
# Group the categories and get their counts
df_health_meta['categories'].apply(lambda x : x[0][0]).value_counts()

Health & Personal Care       262317
CDs & Vinyl                     445
Sports & Outdoors               170
Automotive                       31
Cell Phones & Accessories        21
Home & Kitchen                   15
Baby Products                    11
Electronics                       9
Tools & Home Improvement          6
Office Products                   5
Books                             2
Name: categories, dtype: int64

In [16]:
# Delete product of other categories
rows_to_delete = []
for idx, categories in enumerate(df_health_meta['categories']):
    if categories[0][0] != 'Health & Personal Care':
        rows_to_delete.append(idx)

df_health_meta = df_health_meta.drop(df_health_meta.index[rows_to_delete])
# View size of remaining data 
df_health_meta['categories'].apply(lambda x : x[0][0]).value_counts()

Health & Personal Care    262317
Name: categories, dtype: int64

2.6 Set asin as index

In [21]:
# Check if asin is unique in meta data
print("The asin in health&care meta data is unique: {u}".format(u = df_health_meta['asin'].is_unique))

True

In [22]:
df_health_meta = df_health_meta.set_index(['asin'])

2.7 Generate pickle file

In [23]:
df_health_meta.to_pickle('health_metadata.pkl')

2.8 Get the products' asin.

In [15]:
df_health_meta = pd.read_pickle('health_metadata.pkl')
health_asin = df_health_meta.index.values

### 3. Health&Care reviews
In this part, we begin to turn our attention to Health&Care review dataset.

3.1 Import Health&Care reviews

In [27]:
# Read from compressed data
df_health_review = getDF(DATA_FOLDER + REVIEWS_HEALTH_CARE)

In [ ]:
df_health_review_org = df_health_review.copy()
# Generate pickle for origin review data
df_health_review.to_pickle('health_review_org.pkl')

In [18]:
# Read from pickle
# df_health_review_org = pd.read_pickle('health_review_org.pkl')
# df_health_review = df_health_review_org.copy()

In [19]:
df_health_review.head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0077614992,"[0, 0]",5,This book was a requirement for a college clas...,"02 15, 2013",ARMDSTEI0Z7YW,dodo,great,1360886400
1,0615208479,"[0, 0]",5,This is a great gift for anyone who wants to h...,"12 1, 2008",A3FYN0SZYWN74,Marilyn Mitzel,AMAZING HOW QUICKLY IT WORKS!,1228089600
2,0615269990,"[0, 0]",5,Very helpful in learning about different disea...,"04 6, 2014",A2J0WRZSAAHUAP,icu-rn,Must have,1396742400
3,0615269990,"[1, 1]",5,Occam's Razor is an unbelievably intuitive and...,"12 4, 2013",A38RKP6G5P8J63,JOSEPH A WONG,"Occam's Razor, a game for all ages and backgro...",1386115200
4,0615269990,"[0, 0]",2,I met the guys from Nerdcore at a medical conf...,"04 24, 2014",ARENM677YXZKX,Sonny Patel,Very limited utility,1398297600


In the review dataset, there are nine columns. 

**reviewerID** and **reviewName** are unrelated to our projects, we choose to delete them. 

Also, we notice there are two attributes about time information, **unixReviewTime** which is only numbers and the format of **reviewTime** is not good for view and we don't know whether they are in one time zone, so we decide to keep **unixReviewTime**

3.2  Discard uninterested columns

In [21]:
df_health_review = df_health_review.drop(['reviewerID','reviewerName','reviewTime','summary'],axis=1)

3.3 Change date format to standard datetime

We convert the unix time format into date time format.

In [27]:
df_health_review['unixReviewTime'] = pd.to_datetime(df_health_review['unixReviewTime'],unit='s')
# Reset columns name
df_health_review.rename(columns={'unixReviewTime':'ReviewTime'},inplace=True)
df_health_review.head(1)

,asin,helpful,overall,reviewText,ReviewTime
0,0077614992,"[0, 0]",5,This book was a requirement for a college clas...,2013-02-15


3.4 Check if there exists Nan Value in the DataFrame

The following results show that there isn't any Nan value in the dataframe.

In [29]:
checkDataframeNanValue(df_health_review)

Check if column asin exists Nan value: False
Check if column helpful exists Nan value: False
Check if column overall exists Nan value: False
Check if column reviewText exists Nan value: False
Check if column ReviewTime exists Nan value: False


[]

3.5  Filter out the product using asin which exists in the meta data

In [35]:
df_health_review = df_health_review.loc[df_health_review['asin'].isin(health_asin)]

3.6  Generate pickle file

In [36]:
df_health_review.to_pickle('health_review.pkl')

#### Citation: 
R. He, J. McAuley. Modeling the visual evolution of fashion trends with one-class collaborative filtering. WWW, 2016<br/>
J. McAuley, C. Targett, J. Shi, A. van den Hengel. Image-based recommendations on styles and substitutes. SIGIR, 2015